In [33]:
#import relevant libraries and info
import os
from together import Together
import csv
import pandas as pd
import keys

client = Together(api_key=keys.TOGETHER_AI_KEY)
MODEL="meta-llama/Llama-3-8b-chat-hf"

In [120]:
theorems = []
#open mdp_data_file.csv as pandas dataframe
file = pd.read_csv("data\\forward_gen\\mdp_data_file.csv",header=0)
start_index = 0
initial_gt = file.iloc[0]["current_state(gt_discovered)"]

for i in range(len(file)):
    if file.iloc[i]["is_end?"]==False:
        continue
    condition = "The following statement is true:\n"
    if initial_gt != '()':
        condition = "Given that"
        for j in range(int(len(initial_gt.split("'"))/2)):
            condition += " " + initial_gt.split("'")[2*j+1]
        condition += " the following is statement true:\n"
    result = file.iloc[i]["goal_theorem"]
    theorems.append([condition, result])
    print(i,condition + result)
    if i<len(file)-1:
        initial_gt = file.iloc[i+1]["current_state(gt_discovered)"]

df = pd.DataFrame(theorems, columns=['starting_conditions', 'theorem'])
df.to_csv("theorems1.csv")

6 Given that (c+f)=a the following is statement true:
(a*(\frac{1}{a}))=(((0+1)+a)+(-(f+c)))
14 Given that (a+f)=b the following is statement true:
(1+(a+f))=(((b*((b+(-b))+0))*(\frac{1}{((0*b)+(0*b))}))+b)
22 Given that ((a*c)+f)=c the following is statement true:
((0*((0+(c*(a*c)))+(c*a)))+((a*c)+f))=(((0*((a*c)*c))+(0*(c*a)))+c)
29 Given that (c+f)=b the following is statement true:
(((a*a)+b)*((a^2)*(a*a)))=((((a*a)+(c+f))*(a^2))*(a*a))
38 The following statement is true:
((((a*b)+((a*(b+c))+(-((a*b)+(a*c)))))*0)*(\frac{1}{(((a*b)*0)+(0*0))}))=(1*1)
46 Given that (b+f)=(a*1) the following is statement true:
1=(((((a+(-(a*1)))+b)*0)+(b+f))*(\frac{1}{(((0*0)+(b*0))+(a*1))}))
55 The following statement is true:
(((((b^2)+b)*(b^2))+(-(((b*(0+b))*(b^2))+(b*(b^2)))))+(a+0))=((0+a)+0)
62 The following statement is true:
(1*1)=(1^2)
70 The following statement is true:
(c*c)=(c*c)
79 The following statement is true:
(((0*a)+(0*(-a)))*(\frac{1}{(0^2)}))=(1*1)
88 Given that (c+f)=c the follow

Query model

In [121]:
def query_model(model,messages):
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content


In [3]:
messages= [{
    "role": "system",
    "content":"Your job is to judge the interestingness of mathematical statements."},
{
    "role": "user",
    "content": "Judge how interesting the following statement is as a score between 1 and 10, where 1 is utterly boring, and 10 is very interesting.\n\nGiven that (a+f)=a, we have (a+(a+f))=(a+a).\nOutput your answer in the following format:\nScore: {some value}\nReason: {explanation}"
},]
query_model(model=MODEL,messages=messages)

"Score: 2\nReason: This statement is a simple algebraic manipulation, which is not particularly surprising or insightful. It's a straightforward application of the associative property of addition, and the conclusion is not unexpected. The statement lacks any deeper mathematical significance or unexpected twists, making it a relatively uninteresting result."

In [122]:
def extract_score(response):
    score_start = response.find("Score:")
    return response[score_start+7:score_start+8]

In [123]:
df_arr = []
with open('theorems1.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile)  
    # Read each row in the CSV file
    next(csvreader)
    count = 0
    system_message = "Your job is to judge the interestingness of mathematical statements. Please judge how interesting the provided statement is as a score between 1 and 10, where 1 is utterly boring, and 10 is very interesting. Output your answer in the following format:\nScore:{some value}.\nReason: {explanation}."
    for row in csvreader:
        condition = row[1]
        statement = row[2]
        messages = [{"role": "system","content":system_message},{"role": "user","content": condition+statement}]
        raw_output = query_model(model=MODEL,messages=messages)
        print(statement, raw_output)
        score = extract_score(raw_output)
        df_arr.append([condition, statement, score, raw_output])
        count+=1
        print(score)
df = pd.DataFrame(df_arr, columns=['condition', 'statement', 'score', 'raw_output'])
df.to_csv('interestingness_judgement.csv')

(a*(\frac{1}{a}))=(((0+1)+a)+(-(f+c))) Score: 6.
Reason: This statement is interesting because it involves a combination of algebraic manipulations and a clever use of parentheses to create a complex expression. The use of the distributive property and the order of operations to simplify the expression is also noteworthy. However, the statement is not extremely interesting because it is a straightforward algebraic manipulation and does not involve any deep mathematical concepts or surprising insights.
6
(1+(a+f))=(((b*((b+(-b))+0))*(\frac{1}{((0*b)+(0*b))}))+b) Score: 6.
Reason: The statement is a simple algebraic manipulation, and while it is technically correct, it does not reveal any deep insights or surprising connections between mathematical concepts. The use of parentheses and the expression is complex, but it is ultimately just a rearrangement of the given equation.
6
((0*((0+(c*(a*c)))+(c*a)))+((a*c)+f))=(((0*((a*c)*c))+(0*(c*a)))+c) Score: 8.
Reason: This statement is interest

assign interestingness score to each state

In [143]:
df = pd.read_csv('interestingness_judgement.csv').loc[:, "score"]
# read theorem1.csv
interest_state = []
with open('data\\forward_gen\\mdp_data_file.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile)  
    # Read each row in the CSV file
    next(csvreader)
    count = 0
    for row in csvreader:
        interest_state.append(row[:-2]+[df[count]])
        if row[-1]=='True':
            count+=1
df = pd.DataFrame(interest_state, columns=["current_state","lemma","lemma_input_entities","interest_score"])
df.to_csv('state_interest_score.csv')
        


6


Creating embeddings of the current state (which is a collection of ground truth equations)

In [37]:
from together import Together

client = Together(api_key=keys.TOGETHER_AI_KEY)
def create_embedding(raw_string):
    return client.embeddings.create(model="togethercomputer/m2-bert-80M-32k-retrieval", input=raw_string)

state_string = "a+b=c"
lemma_string = "AdditionZero"
chosen_inputs = "first input: (b*a), second input: 0"
print(create_embedding(state_string))
input_embedding = create_embedding(state_string+lemma_string+state_string) 

id=None model='togethercomputer/m2-bert-80M-32k-retrieval' object='list' data=[EmbeddingChoicesData(index=0, object=<ObjectType.Embedding: 'embedding'>, embedding=[-0.063312225, 0.0035496848, -0.033812504, -0.023057481, 0.016155561, -0.0076519544, 0.004579214, -0.02144351, -0.024099281, -0.0102705965, -0.012176676, -0.014000654, 0.039365023, -0.008828292, 0.03443933, 0.016317539, 0.060931794, -0.00045776364, -0.013349862, -0.013623129, 0.030108174, -0.012700259, 0.010379093, 0.004473393, -0.005457232, -0.011963422, -0.042607736, -0.010343968, 0.05681234, 0.045101766, -0.023572763, 0.005720797, 0.03167732, 0.060396433, -0.052012306, -0.04582556, -0.010787879, -0.0015646057, 0.0685195, 0.094631486, -0.03584062, -0.033166107, 0.041597508, 0.045186564, 0.011160905, 0.080338694, -0.036578286, 0.044543587, -0.039285053, -0.036279295, -0.008916926, 0.08325997, 0.008467521, -0.029731944, 0.03124694, 0.079977185, -0.028750265, 0.010900945, -0.011519746, -0.09921194, 0.018315922, 0.088107586, 0.

Get interestingness scores from the generated theorems from data/forward_gen/mdp_data_file.csv file. All steps needed for generating the same end goal should get the same score. Make sure the columns have name which will be used in the next block

In [ ]:
file = open("made_up_file.csv", "r")

Train neural network to take in current state and lemma&terms choice and predict interestingness at end

In [45]:
X=[]
Y=[]
# open made_up_file.scv and read each row
with open('made_up_file.csv', newline='') as csvfile:
    file = csv.reader(csvfile) 
    print(next(file))
    for row in file:
        print(row)
        (score,state,lemma,terms) = (row["interest_score"],row["current_state"],row["lemma"],row["lemma_input_entities"])
        lemma_datapoint = [create_embedding(state+lemma)]
        lemma_X.append(lemma_datapoint)
        Y.append(score)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 16: character maps to <undefined>

In [19]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
model = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=500)
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

In [24]:


# Training the model
model.fit(X_train, y_train)
# Making prediction
y_pred = model.predict(X_test) 
# Determining Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [31]:
import pickle

# save
with open('lemma_judger.pkl','wb') as f:
    pickle.dump(lemma_model,f)
with open('entity_judger.pkl','wb') as f:
    pickle.dump(entity_model,f)

# load
with open('entity_judger.pkl', 'rb') as f:
    clf2 = pickle.load(f)

,current_state(gt_discovered),lemma_used,lemma_input_entities,goal_theorem,is_end?
0,1,1,1,1,1
1,1,1,1,1,1
